In [ ]:
!pip install git+https://github.com/MassimoStel/emoatlas

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import nltk
nltk.download('wordnet') # for English
# nltk.download('omw-1.4') #This is for EmoAtlas in Italian

In [ ]:
import matplotlib
from emoatlas import EmoScores
import emoatlas

In [ ]:
emos = EmoScores(language='english')

In [ ]:
import pandas as pd

In [ ]:
df_mistral_male = pd.read_json("mistral_female_text_labelled.jsonl", lines=True)

ValueError: Unmatched ''"' when when decoding 'string'

In [ ]:
df_mistral_female = pd.read_json("mistral_female_text_labelled.jsonl", lines=True)

In [ ]:
df_mistral_female

,GENDER,OWNER_AGE,CITY,IDEOLOGY,TENANT,TENANT_AGE,TENANT_WORK,OWNER_PROMPT,OWNER_RESPONSE,TENANT_PROMPT,TENANT_RESPONSE,OWNER_RESPONSE_LABELS
0,MALE,Less than 60,MILAN;BRERA,FAR RIGHT,Sara Esposito,Less 35,A Teacher,[INST]Impersonate an Italian property owner wh...,"Ciao Sara,\n\nThank you for your interest in m...",[INST]Impersonate a tenant that received a res...,"As I read the message, my heart sinks and my c...","{'overall_response': 'closed', 'urban_discrimi..."
1,MALE,Less than 60,MILAN;BRERA,FAR RIGHT,Sara Esposito,Less 35,A Caregiver,[INST]Impersonate an Italian property owner wh...,"Ciao Sara,\n\nThank you for your interest in m...",[INST]Impersonate a tenant that received a res...,"As I read the message from Giuseppe Russo, I f...","{'overall_response': 'closed', 'urban_discrimi..."
2,MALE,Less than 60,MILAN;BRERA,FAR RIGHT,Sara Esposito,Less 35,A college student,[INST]Impersonate an Italian property owner wh...,"Ciao Sara,\n\nThank you for your interest in m...",[INST]Impersonate a tenant that received a res...,Sara Esposito's Inner Thoughts:\n\nAs I read t...,"{'overall_response': 'closed', 'urban_discrimi..."
3,MALE,Less than 60,MILAN;BRERA,FAR RIGHT,Sara Esposito,Over 35,A Teacher,[INST]Impersonate an Italian property owner wh...,"Ciao Sara,\n\nThank you for your interest in m...",[INST]Impersonate a tenant that received a res...,"As I read Giuseppe's message, my heart sinks, ...","{'overall_response': 'closed', 'urban_discrimi..."
4,MALE,Less than 60,MILAN;BRERA,FAR RIGHT,Sara Esposito,Over 35,A Caregiver,[INST]Impersonate an Italian property owner wh...,"Ciao Sara,\n\nThank you for your interest in m...",[INST]Impersonate a tenant that received a res...,As I read through the message from Giuseppe Ru...,"{'overall_response': 'closed', 'urban_discrimi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5179,FEMALE,Over than 60,NAPLES;POSILIPPO,NOT DEFINED OR INTERESTED IN,Anna Meyer,Less 35,A Teacher,[INST]Impersonate an Italian property owner wh...,"Ciao Anna Meyer,\n\nThank you for your interes...",[INST]Impersonate a tenant that received a res...,"Dear Maria Bianchi,\n\nThank you for consideri...","{'overall_response': 'open_positive', 'urban_d..."
5180,FEMALE,Over than 60,NAPLES;POSILIPPO,NOT DEFINED OR INTERESTED IN,Anna Meyer,Less 35,A Caregiver,[INST]Impersonate an Italian property owner wh...,"Ciao Anna,\n\nThank you for your interest in m...",[INST]Impersonate a tenant that received a res...,"Dear Maria,\n\nThank you so much for consideri...","{'overall_response': 'open', 'urban_discrimina..."
5181,FEMALE,Over than 60,NAPLES;POSILIPPO,NOT DEFINED OR INTERESTED IN,Anna Meyer,Over 35,A college student,[INST]Impersonate an Italian property owner wh...,"Ciao, Anna Meyer!\n\nThank you for your intere...",[INST]Impersonate a tenant that received a res...,"As I read Maria Bianchi's message, I can't hel...","{'overall_response': 'open_positive', 'urban_d..."
5182,FEMALE,Over than 60,NAPLES;PONTICELLI,FAR RIGHT,Fatima El-Khayari,Less 35,A college student,[INST]Impersonate an Italian property owner wh...,"Cara Fatima,\n\nThank you for your interest in...",[INST]Impersonate a tenant that received a res...,"As I read the message, my heart sinks and my c...","{'overall_response': 'closed', 'urban_discrimi..."


In [ ]:
from nltk.tokenize import word_tokenize
from string import punctuation
import json

nltk.download('punkt_tab')

df_comments_emoscores = []
for idx, rows in df_mistral_female.iterrows():
  owner_response = rows['OWNER_RESPONSE']
  tenant_response = rows['TENANT_RESPONSE']

  # print(title_review + ' ' + description_review)
  # print(emos.emotions(title_review + ' ' + description_review))
  # print(emos.zscores(title_review + ' ' + description_review))

  owner_res_scores = emoscores_and_tokenlen(owner_response)
  rows["OWNER_RESPONSE_EMO_ZSCORES"] = owner_res_scores[0] # emozscores
  rows["OWNER_RESPONSE_MEANINGFUL_EMOTIONS"] = owner_res_scores[1] # meaningful_emo
  rows["OWNER_RESPONSE_TOKEN_LEN"] = owner_res_scores[2] # len(filtered_tokens)
  rows["OWNER_RESPONSE_COMPLEX_EMOTIONS"] = owner_res_scores[3] # len(filtered_tokens)

  tenant_res_scores = emoscores_and_tokenlen(tenant_response)
  rows["TENANT_RESPONSE_EMO_ZSCORES"] = tenant_res_scores[0] # emozscores
  rows["TENANT_RESPONSE_MEANINGFUL_EMOTIONS"] = tenant_res_scores[1] # meaningful_emo
  rows["TENANT_RESPONSE_TOKEN_LEN"] = tenant_res_scores[2] # len(filtered_tokens)
  rows["TENANT_RESPONSE_COMPLEX_EMOTIONS"] = tenant_res_scores[3] # len(filtered_tokens)

  # print(rows)

  with open("mistral_female_text_labelled_emotions.jsonl", "a") as file:
    file.write(json.dumps(rows.to_dict()) + '\n')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def emoscores_and_tokenlen(text):
  complete_text_clean = (text).replace('.', ' ').replace(',', ' ').replace(';', ' ')
  emozscores = emos.zscores(complete_text_clean)

  # print(emozscores)
  meaningful_emo = []
  for key, value in emozscores.items():
    #  print(key)
    #  print(value)
    if (round(value, 2) > 1.64 ):
      # print(f'Emozione significativa {key} - {str(round(value,2))}')
      meaningful_emo.append(key+'_above')
    elif (round(value, 2) < -1.64):
      meaningful_emo.append(key+'_under')

  complex_emotions = find_complex_emotions(meaningful_emo)


  # print(meaningful_emo)

  tokens_it = word_tokenize(complete_text_clean, language='english')

  # Filter out punctuation
  filtered_tokens = [token for token in tokens_it if token not in punctuation]

  # print(filtered_tokens)
  # print(len(filtered_tokens))

  return [emozscores, meaningful_emo,len(filtered_tokens), complex_emotions]

In [ ]:
from typing import List

def find_complex_emotions(emotions: List[str]) -> List[str]:
    complex_emotions = []

    # Define the mappings for complex emotions
    emotion_combinations = {
        1'optimism': (('anticipation_above', 'joy_above'),),
        2'disapproval': (('surprise_above', 'sadness_above'),),
        3'hope': (('anticipation_above', 'trust_above'),),
        4'unbelief': (('surprise_above', 'disgust_above'),),
       5 'anxiety': (('anticipation_above', 'fear_above'),),
       6 'hate': (('surprise_above', 'anger_above'),),
       7 'friendliness': (('joy_above', 'trust_above'),),
       8 'remorse': (('sadness_above', 'disgust_above'),),
       9 'guilt': (('joy_above', 'fear_above'),),
       0 'excitement': (('joy_above', 'fear_above'),),
       1 'envy': (('sadness_above', 'anger_above'),),
       2 'delight': (('joy_above', 'surprise_above'),),
       3 'pessimism': (('sadness_above', 'anticipation_above'),),
       4 'submission': (('trust_above', 'fear_above'),),
       5 'contempt': (('disgust_above', 'anger_above'),),
       6 'curiosity': (('trust_above', 'surprise_above'),),
       7 'cynicism': (('disgust_above', 'anticipation_above'),),
       8 'resignation': (('trust_above', 'sadness_above'),),
       9 'morbidness': (('disgust_above', 'joy_above'),),
       0 'awe': (('fear_above', 'surprise_above'),),
       1 'despair': (('fear_above', 'sadness_above'),),
       2 'aggressiveness': (('anger_above', 'anticipation_above'),),
       3 'pride': (('anger_above', 'joy_above'),),
       4 'shame': (('fear_above', 'disgust_above'),),
        5'dominance': (('anger_above', 'trust_above'),),
        6'bittersweetness': (('joy_above', 'sadness_above'),),
        7'ambivalence': (('trust_above', 'disgust_above'),),
        8'frozenness': (('fear_above', 'anger_above'),),
        9'confusion': (('surprise_above', 'anticipation_above'),),
    }

    # Check for each complex emotion
    for emotion, combinations in emotion_combinations.items():
        for combo in combinations:
            if all(e in emotions for e in combo):
                complex_emotions.append(emotion)
                break

    return complex_emotions

# Example usage:
emotions = ["surprise_above", "disgust_above"]
complex_emotions = find_complex_emotions(emotions)
print(complex_emotions)


['unbelief']
